In [62]:
import pandas as pd
import numpy as np
import os
import json
import ast
import re
from textblob import TextBlob

In [35]:
# Se lee de cada línea del dataset
filas_review = []
with open(r'C:\Users\ROXANA\Downloads\04-2024\lanzamiento\Roxana\Datasets\australian_user_reviews.json', encoding='utf-8') as f:
    # Se itera sobre cada línea del archivo JSON
    for line in f.readlines():
        # Se utiliza ast.literal_eval para evaluar la línea como una expresión literal de Python (convirtiendo JSON a Python)
        filas_review.append(ast.literal_eval(line))

# Se convierte en dataframe
df_reviews = pd.DataFrame(filas_review)
df_reviews

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."
...,...,...,...
25794,76561198306599751,http://steamcommunity.com/profiles/76561198306...,"[{'funny': '', 'posted': 'Posted May 31.', 'la..."
25795,Ghoustik,http://steamcommunity.com/id/Ghoustik,"[{'funny': '', 'posted': 'Posted June 17.', 'l..."
25796,76561198310819422,http://steamcommunity.com/profiles/76561198310...,"[{'funny': '1 person found this review funny',..."
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"[{'funny': '', 'posted': 'Posted July 21.', 'l..."


In [36]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25799 entries, 0 to 25798
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   user_id   25799 non-null  object
 1   user_url  25799 non-null  object
 2   reviews   25799 non-null  object
dtypes: object(3)
memory usage: 604.8+ KB


In [37]:
# Elimina filas duplicadas en el DataFrame df_reviews basándose en la columna 'user_id'
df_reviews = df_reviews.drop_duplicates(subset='user_id', keep='first')

In [38]:
# Se observa el tipo de dato que contiene 'review'
df_reviews['reviews'][0]

[{'funny': '',
  'posted': 'Posted November 5, 2011.',
  'last_edited': '',
  'item_id': '1250',
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': 'Simple yet with great replayability. In my opinion does "zombie" hordes and team work better than left 4 dead plus has a global leveling system. Alot of down to earth "zombie" splattering fun for the whole family. Amazed this sort of FPS is so rare.'},
 {'funny': '',
  'posted': 'Posted July 15, 2011.',
  'last_edited': '',
  'item_id': '22200',
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': "It's unique and worth a playthrough."},
 {'funny': '',
  'posted': 'Posted April 21, 2011.',
  'last_edited': '',
  'item_id': '43110',
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': 'Great atmosphere. The gunplay can be a bit chunky at times but at the end of the day this game is definitely worth it and I hope they do a sequel...so buy the game so I get a sequel!'}]

In [39]:
# Se transforma a columnas cada elemento de las listas
df_reviews2 = pd.json_normalize(df_reviews['reviews'])
df_reviews2.head()

,0,1,2,3,4,5,6,7,8,9
0,"{'funny': '', 'posted': 'Posted November 5, 20...","{'funny': '', 'posted': 'Posted July 15, 2011....","{'funny': '', 'posted': 'Posted April 21, 2011...",None,None,None,None,None,None,None
1,"{'funny': '', 'posted': 'Posted June 24, 2014....","{'funny': '', 'posted': 'Posted September 8, 2...","{'funny': '', 'posted': 'Posted November 29, 2...",None,None,None,None,None,None,None
2,"{'funny': '', 'posted': 'Posted February 3.', ...","{'funny': '', 'posted': 'Posted December 4, 20...","{'funny': '', 'posted': 'Posted November 3, 20...","{'funny': '', 'posted': 'Posted October 15, 20...","{'funny': '', 'posted': 'Posted October 15, 20...","{'funny': '', 'posted': 'Posted October 15, 20...",None,None,None,None
3,"{'funny': '', 'posted': 'Posted October 14, 20...","{'funny': '', 'posted': 'Posted July 28, 2012....","{'funny': '', 'posted': 'Posted June 2, 2012.'...","{'funny': '', 'posted': 'Posted June 29, 2014....","{'funny': '', 'posted': 'Posted November 22, 2...","{'funny': '', 'posted': 'Posted February 23, 2...",None,None,None,None
4,"{'funny': '3 people found this review funny', ...","{'funny': '1 person found this review funny', ...","{'funny': '2 people found this review funny', ...","{'funny': '', 'posted': 'Posted July 11, 2013....",None,None,None,None,None,None


In [40]:
# Se agrega el 'user_id' y 'user_url' a las columnas separadas
df_reviews2 = pd.concat([df_reviews[['user_id', 'user_url']], df_reviews2], axis=1)
df_reviews2.head()

,user_id,user_url,0,1,2,3,4,5,6,7,8,9
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted November 5, 20...","{'funny': '', 'posted': 'Posted July 15, 2011....","{'funny': '', 'posted': 'Posted April 21, 2011...",None,None,None,None,None,None,None
1,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted June 24, 2014....","{'funny': '', 'posted': 'Posted September 8, 2...","{'funny': '', 'posted': 'Posted November 29, 2...",None,None,None,None,None,None,None
2,evcentric,http://steamcommunity.com/id/evcentric,"{'funny': '', 'posted': 'Posted February 3.', ...","{'funny': '', 'posted': 'Posted December 4, 20...","{'funny': '', 'posted': 'Posted November 3, 20...","{'funny': '', 'posted': 'Posted October 15, 20...","{'funny': '', 'posted': 'Posted October 15, 20...","{'funny': '', 'posted': 'Posted October 15, 20...",None,None,None,None
3,doctr,http://steamcommunity.com/id/doctr,"{'funny': '', 'posted': 'Posted October 14, 20...","{'funny': '', 'posted': 'Posted July 28, 2012....","{'funny': '', 'posted': 'Posted June 2, 2012.'...","{'funny': '', 'posted': 'Posted June 29, 2014....","{'funny': '', 'posted': 'Posted November 22, 2...","{'funny': '', 'posted': 'Posted February 23, 2...",None,None,None,None
4,maplemage,http://steamcommunity.com/id/maplemage,"{'funny': '3 people found this review funny', ...","{'funny': '1 person found this review funny', ...","{'funny': '2 people found this review funny', ...","{'funny': '', 'posted': 'Posted July 11, 2013....",None,None,None,None,None,None


Usamos melt de Pandas para convertir df_reviews2 de ancho a largo, permitiendo descomponer columnas con múltiples valores en nuevas filas. Conservamos user_id y user_url como identificadores únicos para generar un diccionario con la información de cada usuario.

In [41]:
# Las columnas de valor se especifican mediante list(range(9)), y la nueva columna se llama 'reviews'
df_reviews2 = pd.melt(df_reviews2, id_vars=['user_id', 'user_url'],
                       value_vars=list(range(9)),
                       value_name='reviews')

# Muestra las primeras filas del DataFrame df_reviews2 después de la transformación
df_reviews2.head()

,user_id,user_url,variable,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,0,"{'funny': '', 'posted': 'Posted November 5, 20..."
1,js41637,http://steamcommunity.com/id/js41637,0,"{'funny': '', 'posted': 'Posted June 24, 2014...."
2,evcentric,http://steamcommunity.com/id/evcentric,0,"{'funny': '', 'posted': 'Posted February 3.', ..."
3,doctr,http://steamcommunity.com/id/doctr,0,"{'funny': '', 'posted': 'Posted October 14, 20..."
4,maplemage,http://steamcommunity.com/id/maplemage,0,"{'funny': '3 people found this review funny', ..."


In [42]:
# Se eliminan los None en reviews
df_reviews2 = df_reviews2.dropna()
# Se verifica que solo queden el 'user_id' con la cantidad de diccionarios que le corresponde
df_reviews2[df_reviews2['user_id']=='76561197970982479']

,user_id,user_url,variable,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,0,"{'funny': '', 'posted': 'Posted November 5, 20..."
25799,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1,"{'funny': '', 'posted': 'Posted July 15, 2011...."
51598,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2,"{'funny': '', 'posted': 'Posted April 21, 2011..."


In [43]:
#Convertimos cada diccionario en columna
# Se separan por columnas cada una de las claves de 'reviews'
df_reviews = df_reviews2['reviews'].apply(pd.Series, dtype='object')
df_reviews = df_reviews.add_prefix('reviews_')
df_reviews.head()

,reviews_funny,reviews_posted,reviews_last_edited,reviews_item_id,reviews_helpful,reviews_recommend,reviews_review
0,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
2,,Posted February 3.,,248820,No ratings yet,True,A suitably punishing roguelike platformer. Wi...
3,,"Posted October 14, 2013.",,250320,2 of 2 people (100%) found this review helpful,True,This game... is so fun. The fight sequences ha...
4,3 people found this review funny,"Posted April 15, 2014.",,211420,35 of 43 people (81%) found this review helpful,True,Git gud


In [44]:
# Se une con el 'user_id' y 'user_url'
df_reviews = pd.concat([df_reviews2[['user_id', 'user_url']], df_reviews], axis=1)
df_reviews.head()

,user_id,user_url,reviews_funny,reviews_posted,reviews_last_edited,reviews_item_id,reviews_helpful,reviews_recommend,reviews_review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,js41637,http://steamcommunity.com/id/js41637,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
2,evcentric,http://steamcommunity.com/id/evcentric,,Posted February 3.,,248820,No ratings yet,True,A suitably punishing roguelike platformer. Wi...
3,doctr,http://steamcommunity.com/id/doctr,,"Posted October 14, 2013.",,250320,2 of 2 people (100%) found this review helpful,True,This game... is so fun. The fight sequences ha...
4,maplemage,http://steamcommunity.com/id/maplemage,3 people found this review funny,"Posted April 15, 2014.",,211420,35 of 43 people (81%) found this review helpful,True,Git gud


In [45]:
#Reemplazo valores faltanes por nulos
df_reviews.replace('', None, inplace=True)
df_reviews.head()

,user_id,user_url,reviews_funny,reviews_posted,reviews_last_edited,reviews_item_id,reviews_helpful,reviews_recommend,reviews_review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,None,"Posted November 5, 2011.",None,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,js41637,http://steamcommunity.com/id/js41637,None,"Posted June 24, 2014.",None,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
2,evcentric,http://steamcommunity.com/id/evcentric,None,Posted February 3.,None,248820,No ratings yet,True,A suitably punishing roguelike platformer. Wi...
3,doctr,http://steamcommunity.com/id/doctr,None,"Posted October 14, 2013.",None,250320,2 of 2 people (100%) found this review helpful,True,This game... is so fun. The fight sequences ha...
4,maplemage,http://steamcommunity.com/id/maplemage,3 people found this review funny,"Posted April 15, 2014.",None,211420,35 of 43 people (81%) found this review helpful,True,Git gud


In [46]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Index: 57397 entries, 0 to 231501
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   user_id              57397 non-null  object
 1   user_url             57397 non-null  object
 2   reviews_funny        7899 non-null   object
 3   reviews_posted       57397 non-null  object
 4   reviews_last_edited  5898 non-null   object
 5   reviews_item_id      57397 non-null  object
 6   reviews_helpful      57397 non-null  object
 7   reviews_recommend    57397 non-null  bool  
 8   reviews_review       57367 non-null  object
dtypes: bool(1), object(8)
memory usage: 4.0+ MB


In [47]:
# Se eliminan las columnas 'reviews_funny' y 'reviews_last_edited' q tiene valores nulos
df_reviews = df_reviews.drop(columns=['reviews_funny', 'reviews_last_edited'])
df_reviews.columns

Index(['user_id', 'user_url', 'reviews_posted', 'reviews_item_id',
       'reviews_helpful', 'reviews_recommend', 'reviews_review'],
      dtype='object')

In [48]:
df_reviews['reviews_posted'].unique()

array(['Posted November 5, 2011.', 'Posted June 24, 2014.',
       'Posted February 3.', ..., 'Posted January 3, 2011.',
       'Posted December 12, 2010.', 'Posted July 16, 2011.'], dtype=object)

In [49]:
def extraer_anio(fecha_str):
    match = re.search(r'\d{4}', fecha_str)  # Buscar el patrón de cuatro dígitos seguidos, que representa el año
    if match:
        try:
            anio = int(match.group())  # Extraer el año como un número entero
            return anio
        except:
            return None  # Manejar el caso de error de conversión
    else:
        return None  # Manejar el caso de no encontrar el patrón


In [50]:
# Aplicar la función a cada elemento de la serie df_reviews['reviews_posted']
df_reviews['fecha'] = df_reviews['reviews_posted'].apply(extraer_anio)

In [51]:
print(df_reviews['fecha'].unique())

[2011. 2014.   nan 2013. 2015. 2012. 2010.]


In [52]:
print(df_reviews['fecha'].dtype)

float64


In [56]:
num_nan = df_reviews['fecha'].isna().sum()
print("Cantidad de valores NaN en fecha:", num_nan)

Cantidad de valores NaN en fecha: 9771


In [57]:
df_reviews.dropna(subset=['fecha'], inplace=True)
df_reviews['fecha'] = df_reviews['fecha'].astype(int)

In [58]:
print(df_reviews['fecha'].unique())

[2011 2014 2013 2015 2012 2010]


In [59]:
df_reviews.head()

,user_id,user_url,reviews_posted,reviews_item_id,reviews_helpful,reviews_recommend,reviews_review,fecha
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"Posted November 5, 2011.",1250,No ratings yet,True,Simple yet with great replayability. In my opi...,2011
1,js41637,http://steamcommunity.com/id/js41637,"Posted June 24, 2014.",251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,2014
3,doctr,http://steamcommunity.com/id/doctr,"Posted October 14, 2013.",250320,2 of 2 people (100%) found this review helpful,True,This game... is so fun. The fight sequences ha...,2013
4,maplemage,http://steamcommunity.com/id/maplemage,"Posted April 15, 2014.",211420,35 of 43 people (81%) found this review helpful,True,Git gud,2014
5,Wackky,http://steamcommunity.com/id/Wackky,"Posted May 5, 2014.",249130,7 of 8 people (88%) found this review helpful,True,This game is Marvellous.,2014


In [60]:
# Definición de la función de análisis de sentimiento
def analisis_sentimiento(review):  # Verifica si la revisión es None (ninguna).

    if review is None:  # En caso afirmativo, devuelve 1, que podría ser interpretado como un sentimiento neutral.
        return 1

    analysis = TextBlob(review) # Crea una instancia de la clase TextBlob con la revisión proporcionada.
    polarity = analysis.sentiment.polarity  # Obtiene la polaridad del sentimiento del análisis TextBlob.

    if polarity < -0.2:  # Compara la polaridad con umbrales para determinar el sentimiento general.
        return 0  # Si la polaridad es menor que -0.2, se considera un sentimiento negativo y se devuelve 0.

    elif polarity > 0.2:  # Si la polaridad es mayor que 0.2, se considera un sentimiento positivo y se devuelve 2.
        return 2

    else:  # En otros casos, se devuelve 1, que podría ser interpretado como un sentimiento neutral.
        return 1

In [63]:
df_reviews['sentiment_analysis'] = df_reviews['reviews_review'].apply(analisis_sentimiento)
df_reviews.head()

,user_id,user_url,reviews_posted,reviews_item_id,reviews_helpful,reviews_recommend,reviews_review,fecha,sentiment_analysis
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"Posted November 5, 2011.",1250,No ratings yet,True,Simple yet with great replayability. In my opi...,2011,1
1,js41637,http://steamcommunity.com/id/js41637,"Posted June 24, 2014.",251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,2014,1
3,doctr,http://steamcommunity.com/id/doctr,"Posted October 14, 2013.",250320,2 of 2 people (100%) found this review helpful,True,This game... is so fun. The fight sequences ha...,2013,2
4,maplemage,http://steamcommunity.com/id/maplemage,"Posted April 15, 2014.",211420,35 of 43 people (81%) found this review helpful,True,Git gud,2014,1
5,Wackky,http://steamcommunity.com/id/Wackky,"Posted May 5, 2014.",249130,7 of 8 people (88%) found this review helpful,True,This game is Marvellous.,2014,0


In [64]:
df_reviews.to_csv(os.path.join(r'C:\Users\ROXANA\Downloads\04-2024\lanzamiento\Roxana\Datasets', 'reviews_limpios.csv'), index=False)